In [1]:
%pip install transformers

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# %pip install rouge_score

In [3]:
# %pip install nltk

In [4]:
# %pip install torch

In [5]:
#Thực hiện import thư viện và model
import os
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer, AutoTokenizer, AutoModelForSeq2SeqLM
from torch.cuda.amp import GradScaler, autocast

# Kiểm tra GPU
if torch.cuda.is_available():
  device = torch.device("cuda")
  print('Có %d GPU khả dụng.' % torch.cuda.device_count())
  print('Sẽ sử dụng GPU:', torch.cuda.get_device_name(0))
  torch.cuda.empty_cache()
  print("Đã xóa bộ nhớ GPU trước khi tải mô hình")
else:
  print('Không có GPU, sử dụng CPU.')
  device = torch.device("cpu")

model_name = "NlpHUST/t5-small-vi-summarization"
tokenizer = T5Tokenizer.from_pretrained("NlpHUST/t5-small-vi-summarization")

# Load pre-trained model (giữ nguyên để so sánh)
pretrained_model = T5ForConditionalGeneration.from_pretrained(model_name)
pretrained_model.to(device)

# Load checkpoint nếu tồn tại
checkpoint_dir = "Research_T5\\checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)
checkpoint_files = [f for f in os.listdir(checkpoint_dir) if f.startswith("checkpoint_epoch_")]
start_epoch = 0
if checkpoint_files:
  latest_checkpoint = max(checkpoint_files, key=lambda x: int(x.split("_")[-1].split(".")[0]))
  checkpoint_path = os.path.join(checkpoint_dir, latest_checkpoint)
  checkpoint = torch.load(checkpoint_path)
  model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)
  model.load_state_dict(checkpoint['model_state_dict'])
  start_epoch = checkpoint['epoch'] + 1
  print(f"Loaded checkpoint from {checkpoint_path}, resuming from epoch {start_epoch+1}")
else:
  model = T5ForConditionalGeneration.from_pretrained(model_name)
  model.to(device)


src = "Theo BHXH Việt Nam, nhiều doanh nghiệp vẫn chỉ đóng BHXH cho người lao động theo mức lương. Dù quy định từ 1/1/2018, tiền lương tháng đóng BHXH gồm mức lương và thêm khoản bổ sung khác.BHXH Việt Nam vừa có báo cáo về tình hình thực hiện chính sách BHXH thời gian qua.Theo đó, tình trạng nợ, trốn đóng BHXH, BHTN vẫn xảy ra ở hầu hết các tỉnh, thành. Thống kê tới ngày 31/12/2020, tổng số nợ BHXH, BHYT, BHTN là hơn 13.500 tỷ đồng, chiếm 3,35 % số phải thu, trong đó: Số nợ BHXH bắt buộc là hơn 8.600 tỷ đồng, nợ BHTN là 335 tỷ đồng. Liên quan tới tiền lương đóng BHXH, báo cáo của BHXH Việt Nam cho thấy: Nhiều doanh nghiệp vẫn chủ yếu xây dựng thang,bảng lương để đóng BHXH bằng mức thấp nhất. Tức là bằng mức lương tối thiểu vùng, cộng thêm 7 % đối với lao động đã qua đào tạo nghề và cộng thêm 5 % hoặc 7 % đối với lao động làm nghề hoặc công việc nặng nhọc, độc hại, nguy hiểm, đặc biệt nặng nhọc độc hại và nguy hiểm. Đối với lao động giữ chức vụ, khoảng 80 % doanh nghiệp đã xây dựng thang, bảng lương cụ thể theo chức danh. Đơn cử như với chức vụ giám đốc sản xuất, giám đốc điều hành, trưởng phòng. Còn lại các doanh nghiệp xây dựng đối với lao động giữ chức vụ theo thang lương, bảng lương chuyên môn nghiệp vụ và bảng phụ cấp chức vụ, phụ cấp trách nhiệm.Thống kê của BHXH Việt Nam cũng cho thấy, đa số doanh nghiệp đã đăng ký đóng BHXH cho người lao động theo mức lương mà không có khoản bổ sung khác. Mặc dù quy định từ ngày 1/1/2018, tiền lương tháng đóng BHXH gồm mức lương và thêm khoản bổ sung khác."
# Tokenize văn bản đầu vào và chuyển sang tensor PyTorch trên GPU
tokenized_text = tokenizer.encode(src, return_tensors="pt").to(device)

# Đưa mô hình vào chế độ đánh giá (tắt dropout, batchnorm, v.v.)
model.eval()

# Sinh tóm tắt bằng mô hình đã huấn luyện
summary_ids = model.generate(
                    tokenized_text,  # Đầu vào đã được mã hóa
                    max_length=512,  # Độ dài tối đa của đoạn tóm tắt
                    num_beams=5,  # Beam search với 5 beams để cải thiện chất lượng đầu ra
                    repetition_penalty=2.5,  # Phạt lặp từ, giúp tránh việc lặp từ không cần thiết
                    length_penalty=1.0,  # Hệ số phạt độ dài, giá trị >1 sẽ ưu tiên câu dài hơn
                    early_stopping=True  # Dừng sớm nếu tất cả các beam đều tạo ra câu kết thúc
                )

# Giải mã đầu ra của mô hình từ token thành văn bản gốc, bỏ các token đặc biệt
output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(output)


Không có GPU, sử dụng CPU.


ImportError: 
T5Tokenizer requires the SentencePiece library but it was not found in your environment. Checkout the instructions on the
installation page of its repo: https://github.com/google/sentencepiece#installation and follow the ones
that match your environment. Please note that you may need to restart your runtime after installation.


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_checkpoint = "NlpHUST/t5-small-vi-summarization"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

#### Tiền xử lý dữ liệu, chia tập train - test - validation

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer

# Đọc dữ liệu từ excel
df = pd.read_excel('Research_T5\\Dataset\\summary_paper.xlsx', usecols=['body', 'abstract'])

# Load tokenizer
model_name = "NlpHUST/t5-small-vi-summarization"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Hàm tiền xử lý dữ liệu
class SummarizationDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_input_length=512, max_target_length=128):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_input_length = max_input_length
        self.max_target_length = max_target_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        content = self.data.iloc[idx]["body"]
        summary = self.data.iloc[idx]["abstract"]

        # Tokenize dữ liệu đầu vào
        inputs = tokenizer("Tóm tắt: " + content,
                           max_length=self.max_input_length,
                           truncation=True,
                           return_tensors="pt")

        # Tokenize nhãn (tóm tắt)
        labels = tokenizer(summary,
                           max_length=self.max_target_length,
                           truncation=True,
                           return_tensors="pt")

        return {
            "input_ids": inputs["input_ids"].squeeze(),
            "attention_mask": inputs["attention_mask"].squeeze(),
            "labels": labels["input_ids"].squeeze(),
        }

# Collate_fn tùy chỉnh để padding trong batch
def collate_fn(batch, max_input_length=512, max_target_length=128):
    input_ids = [item["input_ids"] for item in batch]
    attention_mask = [item["attention_mask"] for item in batch]
    labels = [item["labels"] for item in batch]

    # Padding động
    input_ids = torch.nn.utils.rnn.pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
    attention_mask = torch.nn.utils.rnn.pad_sequence(attention_mask, batch_first=True, padding_value=0)
    labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=tokenizer.pad_token_id)

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }


# Chia dữ liệu thành tập huấn luyện, kiểm định và kiểm thử (60%-20%-20%)
train_size = int(0.6 * len(df))
val_size = int(0.2 * len(df))

train_df, temp_df = df[:train_size], df[train_size:]
val_df, test_df = temp_df[:val_size], temp_df[val_size:]

print(f"Training data: {len(train_df)} examples")
print(f"Validation data: {len(val_df)} examples")
print(f"Test data: {len(test_df)} examples")

# DataLoader
batch_size = 4
accumulation_steps = 8  # Effective batch_size = 32 (4*8)

train_dataset = SummarizationDataset(train_df, tokenizer, max_input_length=512)
val_dataset = SummarizationDataset(val_df, tokenizer, max_input_length=512)
test_dataset = SummarizationDataset(test_df, tokenizer, max_input_length=512)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                         collate_fn=lambda batch: collate_fn(batch, max_input_length=512, max_target_length=128))
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False,
                       collate_fn=lambda batch: collate_fn(batch, max_input_length=512, max_target_length=128))
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False,
                        collate_fn=lambda batch: collate_fn(batch, max_input_length=512, max_target_length=128))

print(df.head())  # In 5 dòng đầu của DataFrame

In [ ]:
content = df['body'][0]
tokens = tokenizer(content, return_tensors="pt")["input_ids"]
print(tokens.shape[1])

In [ ]:
# Calculate the length of each 'body' and find the maximum
max_length = 0
if 'body' in df.columns:
  max_length = df['body'].str.len().max()
else:
  print("The column 'body' does not exist in the DataFrame.")


print(f"The length of the longest sentence in the dataset is: {max_length}")


#### Bắt đầu fine-tune

In [ ]:
from torch.optim import AdamW

# Optimizer
optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)

if checkpoint_files:
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

In [ ]:
%pip install bert_score

In [ ]:
%pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import numpy as np
from transformers import T5Tokenizer
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from bert_score import score as bert_score
import nltk

# Download NLTK resources if needed
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

# Hàm đánh giá
def evaluate_metrics(model, val_loader, tokenizer, device):
    model.eval()
    predictions, references = [], []
    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Evaluating"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"]
            outputs = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_length=128,
                num_beams=2,
                repetition_penalty=2.5,
                length_penalty=1.0,
                early_stopping=True
            )
            preds = [tokenizer.decode(out, skip_special_tokens=True) for out in outputs]
            refs = [tokenizer.decode(lab, skip_special_tokens=True) for lab in labels]
            predictions.extend(preds)
            references.extend(refs)

    # ROUGE-1, 2, 3, L
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rouge3', 'rougeL'], use_stemmer=True)
    rouge_scores = {"rouge1": [], "rouge2": [], "rouge3": [], "rougeL": []}
    for ref, pred in zip(references, predictions):
        scores = scorer.score(ref, pred)
        for key in rouge_scores:
            rouge_scores[key].append(scores[key].fmeasure)

    # BLEU-1, 2, 3, 4
    smoothie = SmoothingFunction().method1
    bleu_scores = {"bleu1": [], "bleu2": [], "bleu3": [], "bleu4": []}
    for ref, pred in zip(references, predictions):
        ref_tokens = nltk.word_tokenize(ref)
        pred_tokens = nltk.word_tokenize(pred)
        bleu_scores["bleu1"].append(sentence_bleu([ref_tokens], pred_tokens, weights=(1, 0, 0, 0), smoothing_function=smoothie))
        bleu_scores["bleu2"].append(sentence_bleu([ref_tokens], pred_tokens, weights=(0, 1, 0, 0), smoothing_function=smoothie))
        bleu_scores["bleu3"].append(sentence_bleu([ref_tokens], pred_tokens, weights=(0, 0, 1, 0), smoothing_function=smoothie))
        bleu_scores["bleu4"].append(sentence_bleu([ref_tokens], pred_tokens, weights=(0, 0, 0, 1), smoothing_function=smoothie))

    # BERTScore
    P, R, F1 = bert_score(predictions, references, lang="vi", verbose=True)
    bertscore_f1 = F1.mean().item()

    # Tổng hợp kết quả
    results = {
        "ROUGE-1": sum(rouge_scores["rouge1"]) / len(rouge_scores["rouge1"]),
        "ROUGE-2": sum(rouge_scores["rouge2"]) / len(rouge_scores["rouge2"]),
        "ROUGE-3": sum(rouge_scores["rouge3"]) / len(rouge_scores["rouge3"]),
        "ROUGE-L": sum(rouge_scores["rougeL"]) / len(rouge_scores["rougeL"]),
        "BLEU-1": sum(bleu_scores["bleu1"]) / len(bleu_scores["bleu1"]),
        "BLEU-2": sum(bleu_scores["bleu2"]) / len(bleu_scores["bleu2"]),
        "BLEU-3": sum(bleu_scores["bleu3"]) / len(bleu_scores["bleu3"]),
        "BLEU-4": sum(bleu_scores["bleu4"]) / len(bleu_scores["bleu4"]),
        "BERTScore": bertscore_f1
    }
    return results

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

In [ ]:
if torch.cuda.is_available():
    print(f"Bộ nhớ đã cấp: {torch.cuda.memory_allocated(device) / 1024**3:.2f} GiB")
    print(f"Bộ nhớ giữ: {torch.cuda.memory_reserved(device) / 1024**3:.2f} GiB")

In [ ]:
import matplotlib.pyplot as plt
from tqdm import tqdm

# Hàm lưu checkpoint
def save_checkpoint(model, optimizer, epoch, checkpoint_dir):
    os.makedirs(checkpoint_dir, exist_ok=True)  # Tạo thư mục nếu chưa có
    checkpoint_path = os.path.join(checkpoint_dir, f"checkpoint_epoch_{epoch+1}.pt")
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }, checkpoint_path)

    print(f"Đã lưu checkpoint tại {checkpoint_path}")

# Hàm huấn luyện có checkpoint
def train_model(model, pretrained_model, train_loader, val_loader, test_loader, optimizer, device, epochs=10, checkpoint_dir=checkpoint_dir, start_epoch=0):
    best_val_loss = float("inf")
    patience = 5
    patience_counter = 0
    train_losses = []
    val_losses = []

    '''
    # Xóa bộ nhớ GPU trước khi bắt đầu huấn luyện
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        print("Deleted GPU memory before training")
    '''

    # Tính tổng số bước trong một epoch (train + val)
    total_steps_per_epoch = len(train_loader) + len(val_loader)

    # Đánh giá mô hình pre-trained trên tập test trước khi fine-tune
    print("Evaluating Pre-trained Model on Test set...")
    pretrained_metrics = evaluate_metrics(pretrained_model, test_loader, tokenizer, device)
    print("Pre-trained Metrics:")
    for metric, value in pretrained_metrics.items():
        print(f"{metric}: {value:.4f}")

    '''
    # Xóa bộ nhớ GPU sau khi đánh giá mô hình pre-trained
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        print("Deleted GPU memory after evaluating the pre-trained model")
    '''
    # Bắt đầu fine-tune
    for epoch in range(start_epoch, epochs):
        model.train()
        train_loss = 0
        optimizer.zero_grad()

        # Một thanh tqdm duy nhất cho cả train và val
        with tqdm(total=100, desc=f"Epoch {epoch+1}/{epochs}", unit="%", bar_format="{l_bar}{bar}| {n:.0f}/{total:.0f}% [{elapsed}<{remaining}]") as pbar:
            # Train phase
            steps_completed = 0
            for i, batch in enumerate(train_loader):
                input_ids = batch["input_ids"].to(device)
                attention_mask = batch["attention_mask"].to(device)
                labels = batch["labels"].to(device)
                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss / accumulation_steps  # Chia nhỏ loss theo accumulation_steps
                loss.backward()
                if (i + 1) % accumulation_steps == 0:  # Cập nhật trọng số sau mỗi accumulation_steps
                    optimizer.step()
                    optimizer.zero_grad()
                train_loss += loss.item() * accumulation_steps

                # Cập nhật tiến trình
                steps_completed += 1
                progress = (steps_completed / total_steps_per_epoch) * 100
                pbar.n = progress
                pbar.refresh()

            avg_train_loss = train_loss / len(train_loader)
            train_losses.append(avg_train_loss)

            # Val phase
            model.eval()
            val_loss = 0
            with torch.no_grad():
                for batch in val_loader:
                    input_ids = batch["input_ids"].to(device)
                    attention_mask = batch["attention_mask"].to(device)
                    labels = batch["labels"].to(device)
                    outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                    val_loss += outputs.loss.item()

                    # Cập nhật tiến trình
                    steps_completed += 1
                    progress = (steps_completed / total_steps_per_epoch) * 100
                    pbar.n = progress
                    pbar.refresh()

            avg_val_loss = val_loss / len(val_loader)
            val_losses.append(avg_val_loss)

            # Đánh giá metrics trên val set
            metrics = evaluate_metrics(model, val_loader, tokenizer, device)
            print(f"\nEpoch {epoch+1}: Train Loss = {avg_train_loss:.4f}, Val Loss = {avg_val_loss:.4f}, Metrics: {metrics}")

        # Lưu checkpoint và xóa cache
        save_checkpoint(model, optimizer, epoch, checkpoint_dir)
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            print(f"Đã xóa cache GPU sau epoch {epoch+1}")

        # Early stopping và lưu mô hình tốt nhất
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), os.path.join(checkpoint_dir, "best_model.pt"))
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break
    # Đánh giá mô hình fine-tuned trên tập test
    print("Evaluating Fine-tuned Model on Test set...")
    finetuned_metrics = evaluate_metrics(model, test_loader, tokenizer, device)
    print("Fine-tuned Metrics:")
    for metric, value in finetuned_metrics.items():
        print(f"{metric}: {value:.4f}")

    # Vẽ biểu đồ Train loss và Val loss
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, len(train_losses) + 1), train_losses, label="Train Loss", marker='o')
    plt.plot(range(1, len(val_losses) + 1), val_losses, label="Val Loss", marker='o')
    plt.title("Train Loss và Val Loss qua các Epoch")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.grid(True)
    plt.show()

    print("\nSo sánh Pre-trained vs Fine-tuned:")
    print("{:<15} {:<15} {:<15}".format("Chỉ số", "Pre-trained", "Fine-tuned"))
    for metric in pretrained_metrics.keys():
        print("{:<15} {:<15.4f} {:<15.4f}".format(metric, pretrained_metrics[metric], finetuned_metrics[metric]))

    return pretrained_metrics, finetuned_metrics

In [ ]:
# Chạy quá trình fine-tune
pretrained_metrics, finetuned_metrics = train_model(model, pretrained_model, train_loader, val_loader, test_loader, optimizer, device, epochs=10, checkpoint_dir=checkpoint_dir, start_epoch=start_epoch)

Epoch 1/10:  23%|██▎       | 23/100% [03:13<10:45]

In [ ]:
def summarize(text, model, tokenizer, device):
    model.eval()
    inputs = tokenizer("Tóm tắt: " + text, return_tensors="pt", max_length=512, truncation=True).to(device)
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)
    with torch.no_grad():
        summary_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=128, num_beams=4, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Test với một câu
test_text = df.iloc[0]["body"]
print("Tóm tắt:", summarize(test_text, model, tokenizer, device))
